<a href="https://colab.research.google.com/github/Ryansyaaw/Machine-Learning_2023/blob/main/week%2010/tugas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import TensorFlow

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

Download Dataset Shakespeare

In [2]:
path_to_file=tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


Load Data

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


Olah Teks

Vectorize Teks

pecah teks menjadi token

In [6]:
example_texts=['abcdefg','xyz']
chars=tf.strings.unicode_split(example_texts,input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

buat tf.keras.layers.StringLookup layer

In [7]:
ids_from_chars=tf.keras.layers.StringLookup(vocabulary=list(vocab),mask_token=None)

perintah diatas mengconvert token menjadi id

In [8]:
ids=ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Karena tujuan tutorial ini adalah untuk menghasilkan teks, penting juga untuk membalikkan representasi ini. Untuk ini Anda dapat menggunakan kode tf.keras.layers.StringLookup(..., invert=True).

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

Lapisan ini mengconvert kembali karakter dari vektor ID, dan mengembalikannya sebagai karakter tf.RaggedTensor

In [10]:
chars=chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

menggunakan tf.strings.reduce_join untuk menggabungkan kembali karakter menjadi string.


In [11]:
tf.strings.reduce_join(chars,axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

prediksi


Membuat Trianing Set dan Target

In [13]:
all_ids =ids_from_chars(tf.strings.unicode_split(text,'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [14]:
ids_dataset=tf.data.Dataset.from_tensor_slices(all_ids)

In [15]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [16]:
seq_length=100

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [18]:
def split_input_target(sequence):
    input_text=sequence[:-1]
    target_text=sequence[1:]
    return input_text,target_text

In [19]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [20]:
dataset = sequences.map(split_input_target)

In [21]:
for input_example,target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


Membuat Batch Training

In [22]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

Buat Model

In [23]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [24]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [25]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

Uji Model

In [26]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [27]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [28]:
sampled_indices = tf.random.categorical(example_batch_predictions[0],num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [29]:
sampled_indices

array([11, 52, 44, 29, 10, 34, 48,  3,  1, 59, 59, 40, 40, 61, 55, 54, 47,
        8, 43, 26, 35,  5, 21, 50,  1, 31, 24, 10, 21, 51, 64, 41,  1, 11,
       62,  5, 34, 65, 58,  3, 15,  4, 21, 62, 29, 62, 45, 64, 46, 48, 37,
       47, 59, 11, 41,  8, 39, 21, 33, 41, 40, 40, 55, 16, 64, 49, 23, 61,
       63, 60, 27, 49, 21, 31, 45, 65, 44, 22, 19, 64, 57, 37, 31, 27, 26,
       23, 37, 52, 47,  3, 17, 25, 38, 38, 22, 46, 30, 10, 53, 26])

In [30]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"tisfied: I'll read enough,\nWhen I do see the very book indeed\nWhere all my sins are writ, and that's"

Next Char Predictions:
 b':meP3Ui!\nttaavpoh-dMV&Hk\nRK3Hlyb\n:w&Uzs!B$HwPwfygiXht:b-ZHTbaapCyjJvxuNjHRfzeIFyrXRNMJXmh!DLYYIgQ3nM'


**Train Model**

Tambahan optimizer dan fungsi loss

In [31]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [32]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1898627, shape=(), dtype=float32)


In [33]:
tf.exp(example_batch_mean_loss).numpy()

66.013725

In [34]:
model.compile(optimizer='adam',loss=loss)

Konfigurasi Checkpoints

In [35]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Lakukan Proses Training

In [36]:
EPOCHS=20
history=model.fit(dataset,epochs=EPOCHS,callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 20s 59ms/step - loss: 2.7416
Epoch 2/20
172/172 [==============================] - 12s 55ms/step - loss: 1.9995
Epoch 3/20
172/172 [==============================] - 12s 56ms/step - loss: 1.7226
Epoch 4/20
172/172 [==============================] - 11s 56ms/step - loss: 1.5576
Epoch 5/20
172/172 [==============================] - 11s 56ms/step - loss: 1.4579
Epoch 6/20
172/172 [==============================] - 12s 59ms/step - loss: 1.3875
Epoch 7/20
172/172 [==============================] - 11s 56ms/step - loss: 1.3349
Epoch 8/20
172/172 [==============================] - 12s 57ms/step - loss: 1.2894
Epoch 9/20
172/172 [==============================] - 12s 58ms/step - loss: 1.2483
Epoch 10/20
172/172 [==============================] - 12s 60ms/step - loss: 1.2084
Epoch 11/20
172/172 [==============================] - 12s 58ms/step - loss: 1.1686
Epoch 12/20
172/172 [==============================] - 13s 59ms/step - loss: 1.1272
E

Generate Teks

In [37]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [38]:
one_step_model=OneStep(model,chars_from_ids,ids_from_chars)

In [39]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
If whom thou know'st, woe speak thou! Tratious in my belly
And do not pull them with her speech, to enter penalty,
years, the field, do high HOrr'dour thunder;
I am the very ring through the wounds
Nor now mean to her else,--for informate's sceptre's
wailing, to-night she would post unto,
Look in a pile, and fires for truthmen: I thought,
When I were damned leads me from his body
And court of danger, and three parts
That such a shamble-tatch fear.

DUKE OF YORK:
He will know with him? If revenge
Tull's fair Siching the swords become my childness nothing else
to dincerous eyes to see her after? at least
Hath true the first quiet surpect their own virtue.
Thine eyes, let them all read.

HASTINGS:
Men your brother is to death.
Can I never saw I never saw'd:
Daily dedicate be thy knife to thee;
The loving sidet clean us in a raves-loud thing.

DUKE OF AUMERLE:
Yes; I'll none impose him, go to
him up who abul again,
He being more reckened to their success.

GLOUCESTER:
But come to my

In [40]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nAnd should not burne again unto the Tower,\nOn purp he rid Henry, we must suggester\nGo walk: despise and thence then,\nSince thou art woo'd with the charge,\nTo keep the idunerate; at away.\n\nDUKE OF YORK:\nThou constants, monarnthy and mine, and at my master's house:\nThe one perfect I have done. Call help.\nO mischance of breath I paintent to awaken men,\nThat know'st thou makes evil my penspecoal\nOf tyranny. This fellow befiting wome\nThan forth of love!\nBut welcome, dear saint,\nAnd I'll not have showed again Bushy and I ap\ntheir love oir streets against the servants, there,\nWill owe my hands my men distiven blessed.\nThinking on the bellying Bohemia shall,\nOne Warwick and the father, for that mutiny\nHeart if you call feel thy formers and my pack;\nI'll bear my colour's nest, Blacknond\nClord's temple: I beseech you, fear thus.\n\nHERMIONE:\nWhat's your well?\n\nMAMILLIUS:\nI am peter'd of this.\n\nDUKE VINCENTIO:\nDo she such a court, he knowing dows.

Ekspor Model Generator

In [41]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [42]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
If you shall know this way?

MENENIUS:
Not my good labour,
Pinceting his advice: my faith in needle


## **Tugas Praktikum**

In [43]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [44]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [45]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [46]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 15s 59ms/step - loss: 2.7135


In [47]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1801
Epoch 1 Batch 50 Loss 2.0368
Epoch 1 Batch 100 Loss 1.9778
Epoch 1 Batch 150 Loss 1.8827

Epoch 1 Loss: 1.9826
Time taken for 1 epoch 13.44 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8150
Epoch 2 Batch 50 Loss 1.7373
Epoch 2 Batch 100 Loss 1.7190
Epoch 2 Batch 150 Loss 1.6384

Epoch 2 Loss: 1.7026
Time taken for 1 epoch 11.00 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5754
Epoch 3 Batch 50 Loss 1.5611
Epoch 3 Batch 100 Loss 1.5332
Epoch 3 Batch 150 Loss 1.4659

Epoch 3 Loss: 1.5447
Time taken for 1 epoch 11.04 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4789
Epoch 4 Batch 50 Loss 1.4824
Epoch 4 Batch 100 Loss 1.4603
Epoch 4 Batch 150 Loss 1.4381

Epoch 4 Loss: 1.4468
Time taken for 1 epoch 11.15 sec
_____________________________________________________________________